In [ ]:
# Leo's

import msprime
import tskit
import random
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# --- Demographic model ---

def history_of_mexicans(len_seq, T, t_mig, N_e, n ):
    demography = msprime.Demography()
    
    demography.add_population(name="AF", initial_size=500000)
    demography.add_population(name="EU", initial_size=300000)
    demography.add_population(name="NA", initial_size=40000)
    demography.add_population(name="MX", initial_size=40000)

    demography.add_population(name="OUT_AF", initial_size=N_e)
    demography.add_population(name="AF_ANC", initial_size=N_e)

    demography.add_population(name="NEAND", initial_size=30000)
    demography.add_population(name="ANCES", initial_size=N_e)




    demography.add_admixture(time= T[3] , derived="MX", ancestral=["AF", "EU", "NA"], proportions=[0.05, 0.5, 0.45])
    demography.add_population_split(time=T[2], derived=["EU", "NA"], ancestral="OUT_AF")
    demography.add_mass_migration(time=t_mig, source='OUT_AF', dest='NEAND', proportion=0.05)
    demography.add_population_split(time=T[1], derived=["AF", "OUT_AF"], ancestral="AF_ANC")
    demography.add_population_split(time=T[0], derived=["AF_ANC", "NEAND"], ancestral="ANCES")

    TS = msprime.sim_ancestry(
        samples=
        [       
                msprime.SampleSet(1, ploidy=1, population='MX'),
                msprime.SampleSet(n, ploidy=1, population='EU'), 
                msprime.SampleSet(n, ploidy=1, population='NA'),
                msprime.SampleSet(n, ploidy=1, population='AF'),
                msprime.SampleSet(25, ploidy=1, population='NEAND', time=2000)          
               
        ],
    
        ploidy=1,    
        sequence_length=len_seq,
        recombination_rate=2.5e-9, 
        demography=demography,
        record_migrations=True, 
        random_seed=54321299,
        num_replicates = 15
                                
    )
    prop = []
    for replicate_index, ts in enumerate(TS):
        INDIVIDUAL = -1
        for i in ts.tables.migrations:
            if i.time == T[3] and i.dest == 0:
                INDIVIDUAL = i.node
                
            if INDIVIDUAL != -1: 
                
                proportions = np.array([0,  0, 0]) #proportions Af, Eu, NA
                for i in ts.tables.migrations:
                    if i.time == T[3] and i.node == INDIVIDUAL:
                        if i.dest == 0:
                            proportions[0] += i.right - i.left
                        if i.dest == 1:
                            proportions[1] += i.right - i.left
                        if i.dest == 2:
                            proportions[2] += i.right - i.left
                proportions = proportions / sum(proportions)
                break
        if INDIVIDUAL != -1:
            prop.append([proportions, ts])
    
    
    s = 1
    for i in range(len(prop)):
        if prop[i][ 0][0] < 0.1 and prop[i][ 0][0] > 0.01 and prop[i][0][2] > 0.2 and prop[i][0][1] > 0.3:
            s = prop[i][0][0]
            ts_af = prop[i][1]
            proportions = prop[i][0]
            break
    print('We generate mexican individual with Af-Eu-NA proportions = ', proportions)
    
    ts_af = msprime.sim_mutations(ts_af, rate=1.25e-8, random_seed=4321994)
    
    return ts_af, proportions

In [ ]:


# --- Functions related to HMM ---

def initS(a,b,c) -> np.array: # Eu, NA, Nd
    S = np.zeros(5)
    S[0]=a*(1-c)
    S[1]=a*c
    S[2]=b*(1-c)
    S[3]=b*c
    S[4]=1-a-b
    return S

#Ti: Introgression of Nd
#Tmex: Time in Mexico
def initA(Ti,Tmex,r,L,a,b,c) -> np.array:
    A = np.zeros((5,5))
    d=1-a-b
    A[0][1]=Ti*r*L*a*c
    A[0][2]=Tmex*r*L*b*(1-c)
    A[0][3]=Tmex*r*L*b*c
    A[0][4]=Tmex*r*L*d
    A[0][0]=1-A[0][1]-A[0][2]-A[0][3]-A[0][4]
 
    A[1][0]=Ti*r*L*a*(1-c)
    A[1][2]=Tmex*r*L*b*(1-c)
    A[1][3]=Tmex*r*L*b*c
    A[1][4]=Tmex*r*L*d
    A[1][1]=1-A[1][0]-A[1][2]-A[1][3]-A[1][4]
    
    A[2][0]=Tmex*r*L*a*(1-c)
    A[2][1]=Tmex*r*L*a*c
    A[2][3]=Ti*r*L*b*c
    A[2][4]=Tmex*r*L*d
    A[2][2]=1-A[2][0]-A[2][1]-A[2][3]-A[2][4]
    
    A[3][0]=Tmex*r*L*a*(1-c)
    A[3][1]=Tmex*r*L*a*c
    A[3][2]=Ti*r*L*b*(1-c)
    A[3][4]=Tmex*r*L*d
    A[3][3]=1-A[3][0]-A[3][1]-A[3][2]-A[3][4]

    A[4][0]=Tmex*r*L*a*(1-c)
    A[4][1]=Tmex*r*L*a*c
    A[4][2]=Tmex*r*L*b*(1-c)
    A[4][3]=Tmex*r*L*b*c
    A[4][4]=1-A[4][0]-A[4][1]-A[4][2]-A[4][3]
    
    return A

#Ti: Introgression of Nd
#Tea: Split between Asia and Europe
#Tmex: Time of migration to Mexixo
#Taf: Time out of Africa
#Tn: Time of Split between Nd and Sapiens

def initB(m,L,Ti,Tea,Tmex,Taf,Tn, n_st) -> np.array: 
    print("Start Init B")
    B = np.empty(shape=(5,n_st,n_st,n_st,n_st))
    meani = 2*m*L*Ti
    meanea = 2*m*L*Tea
    meanmex = 2*m*L*Tmex
    meanaf = 2*m*L*Taf
    meann = 2*m*L*Tn

    
    Pi = np.empty(n_st)
    Pea = np.empty(n_st)
    Pmex = np.empty(n_st)
    Paf=np.empty(n_st)
    Pn=np.empty(n_st)

    
    Pi[0]=np.exp(-meani)
    Pea[0]=np.exp(-meanea)
    Pmex[0]=np.exp(-meanmex)
    Paf[0]=np.exp(-meanaf)
    Pn[0]=np.exp(-meann)
    
    sumi=0
    sumea=0
    summex=0
    sumaf=0
    sumn=0
    
    for i in range(1,n_st):
        Pi[i]=Pi[i-1]*meani/i
        Pea[i]=Pea[i-1]*meanea/i
        Pmex[i]=Pmex[i-1]*meanmex/i
        Paf[i]=Paf[i-1]*meanaf/i
        Pn[i]=Pn[i-1]*meann/i
        
        sumi=sumi+Pi[i]
        sumea=sumea+Pea[i]
        summex=summex+Pmex[i]
        sumaf=sumaf+Paf[i]
        sumn=sumn+Pn[i]

    Pea[0]=1-sumea
    Pi[0]=1-sumi
    Pmex[0]=1-summex
    Paf[0]=1-sumaf
    Pn[0]=1-sumn
    
    for i in range(n_st): 
        for j in range(n_st):
            for k in range(n_st):
                for t in range(n_st):
                    B[0][i][j][k][t]=Pmex[i]*Pea[j]*Paf[k]*Pn[t]
                    B[1][i][j][k][t]=Pmex[i]*Pea[j]*Pn[k]*Pi[t]
                    B[2][i][j][k][t]=Pea[i]*Pmex[j]*Paf[k]*Pn[t]
                    B[3][i][j][k][t]=Pea[i]*Pmex[j]*Pn[k]*Pi[t]
                    B[4][i][j][k][t]=Paf[i]*Paf[j]*Pmex[k]*Pn[t]    
    return B

In [ ]:


            
def viterbi(V, initial_distribution, a, b):
    
    T = len(V)
    M = a.shape[0]
 
    omega = np.zeros((T, M))
    omega[0, :] = np.log(initial_distribution * b[:, V[0][0],V[0][1],V[0][2],V[0][3]])
 
    prev = np.zeros((T - 1, M))
 
    for t in range(1, T):
        for j in range(M):
            # Same as Forward Probability
            probability = omega[t - 1] + np.log(a[:, j]) + np.log(b[j, V[t][0], V[t][1], V[t][2],V[t][3]])
 
            # This is our most probable state given previous state at time t (1)
            prev[t - 1, j] = np.argmax(probability)
 
            # This is the probability of the most probable state (2)
            omega[t, j] = np.max(probability)
 
    # Path Array
    S = np.zeros(T)
 
    # Find the most probable last hidden state
    last_state = np.argmax(omega[T - 1, :])
 
    S[0] = last_state
 
    backtrack_index = 1
    for i in range(T - 2, -1, -1):
        S[backtrack_index] = prev[i, int(last_state)]
        last_state = prev[i, int(last_state)]
        backtrack_index += 1
 
    # Flip the path array since we were backtracking
    S = np.flip(S, axis=0)
 
    # Convert numeric values to actual hidden states
 
    result = []
    for s in S:
        if s == 0:
            result.append(0)
        elif s == 1:
            result.append(1)
        elif s == 2:
            result.append(2)
        elif s == 3:
            result.append(3)
        elif s == 4:
            result.append(4)

    return result

# --- Other functions ---

# Creates the sequence of observations.
def createSeqObsS(ts,cut,ind,pop) -> list:
    tables = ts.dump_tables()
    nodes = tables.nodes
    seq = np.zeros(int(ts.sequence_length/cut),dtype=int)  #The list which will contain the result of our function.
    pop_id = [p.id for p in ts.populations() if p.metadata['name']==pop][0] 
    start=-1
    end=-1
    for i in range(len(nodes)):
        if nodes[i].population==pop_id and start ==-1:
            start = i
        if nodes[i].population!=pop_id and start !=-1:
            end = i-1
            break
    
    for v in ts.variants():
        i=int(v.site.position/cut)
        c = False
        x=0
        while x < end: 
            if(nodes[x].individual==ind): # Check is this position corresponds to individual 'ind'
                b=v.genotypes[x]
                
                x=start
            elif(nodes[x].population==pop_id): # Check is this position corresponds to an individual in 'pop'
                
                if(v.genotypes[x]==b): 
                    c = True # The mutation found in 'ind' is also present in the population 'pop' 
                    x = end # We can skip all the remaining individuals
            x=x+1                     
        if not c : # If c is False it means that the mutation in 'ind' has not been found in the population 'pop' 
            seq[i]=seq[i]+1
    return seq



def tracts_to_states(len_seq, tractsND,tractsEU,tractsAS,tractsAF):
    seqTrue=np.zeros(len_seq, dtype=int)
    for t in tractsND:
        for i in range(t[0],t[1]):
            seqTrue[i]=1
    for t in tractsAS:
        for i in range(t[0],t[1]):
            if seqTrue[i]==0:
                seqTrue[i]=2
            else:
                seqTrue[i]=3
    for t in tractsAF:
        for i in range(t[0],t[1]):
            seqTrue[i]=4    
    return seqTrue


def tracts_to_states_after_vit(len_seq, tracts):
    seqTrue=np.zeros(len_seq, dtype=int)
    for j in range(5):
        
        for t in tracts[j]:
            for i in range(t[0],t[1]):
                seqTrue[i]=j
 
    return seqTrue



def get_HMM_tracts(seq):
    migrating_tracts = []
    for i in range(5):
        migrating_tracts.append([])
    start=0
    for i in range(1,len(seq)):
        if seq[i]!=seq[i-1]:
            migrating_tracts[seq[i-1]].append([start,i-1])
            start=i
    migrating_tracts[seq[len(seq)-1]].append([start,len(seq)-1])
    return migrating_tracts



In [ ]:

def clean_tracts(tractInit, CUT):
    tract = np.copy(tractInit)
    tract = tract/CUT
    tract=tract.astype(int)
    flag = True
    while(flag):
        flag=False
        for i in range(len(tract)):
            for j in range(len(tract)):
                if not flag and tract[i,0]==tract[j,1]:
                    tract[j,1]=tract[i,1]
                    tract = np.delete(tract,i,0)
                    flag=True
    flag = True
    while(flag):
        flag=False
        for i in range(len(tract)):
            for j in range(i+1,len(tract)):
                if tract[i,0]>tract[j,0]:
                    save0=tract[i,0]
                    save1=tract[i,1]
                    tract[i,0]=tract[j,0]
                    tract[i,1]=tract[j,1]
                    tract[j,0]=save0
                    tract[j,1]=save1
                    flag=True
    return tract

      

#несколько вспомогательных функций
def connected(m):
    for i in range(len(m)-1):
        if m[i][1] == m[i+1][0]:
            return True
    return False
        
def remove_one(m):
    mas = m
    while connected(mas) == True:
        for i in range(len(mas)-1):
            if mas[i][1] == mas[i+1][0]:
                mas[i][1] = mas[i+1][1]
                mas.pop(i+1)
                break
    return mas


#Вход: ts, название популяции, индивид(которого мы препарируем), время предка
def get_migrating_tracts_ind(ts, pop, ind, T_anc):
    mig = ts.tables.migrations
    migration_int = []

    for tree in ts.trees():  #перебираем все деревья. Как известно, каждому дереву отвечает участок днк  
        anc_node = ind #выбираем мексиканца
        while tree.time( tree.parent(anc_node) ) <= T_anc : #идем в прошлое до вершины anc_node по предкам нашего мексиканца, пока не наткнемся на миграцию неандертальцев
            anc_node = tree.parent(anc_node)
        migs = np.where(mig.node == anc_node)[0] #выбирем все строки, соответствующие заданному узлу

        #идем по таблице миграций с anc_node и проверяем, чтобы миграции попадали в тот самый участок днк
        for i in migs:

            stroka = mig[i]
            if stroka.time == T_anc and stroka.dest == pop and tree.interval.left >= stroka.left and tree.interval.right <= stroka.right:
                migration_int.append([tree.interval.left, tree.interval.right])

    migration_int2 = []
    for i in range(len(migration_int)):
        if migration_int[i][0] != migration_int[i][1]:
            migration_int2.append(migration_int[i])
    migration_int = migration_int2
    
    mi = remove_one(migration_int)
    mi.sort()  

    return mi



#пересекаются ли интервалы. -1==нет
def two_intersection(i1, i2):
    if i1[0] > i2[1] or i1[1] < i2[0]:
        return -1
    if i1[0] >= i2[0] and i1[0] <= i2[1] and i1[1] >= i2[1]:
        return [i1[0], i2[1]]
    if i1[0] <= i2[0] and i1[1] <= i2[1] and i1[1] >= i2[0]:
        return [i2[0],i1[1]]
    if (i1[0] <= i2[0] and i1[1] >= i2[1]) or  (i2[0] <= i1[0] and i2[1] >= i1[1]):
        return [max(i1[0],i2[0]), min(i1[1], i2[1])]
    
    
#для двух наборов интервалов ищется их пересечение    
def intersections(l1 , l2):
    s = []
    for i in l1:
        for j in l2:
            int_ij = two_intersection(i, j) 
            if int_ij != -1 and int_ij[0]!=int_ij[1]:
                s.append(int_ij)

    return s

# лежит ли точка p в интервале i?
def point_in_interval(p, i):
    if p >= i[0] and p <= i[1]:
        return True
    else:
        return False
    
# возвращает раскраску по состояниям в каждой позиции, тракты и [пропорции соврем днк в европейце, в американце ]

def coloring(ts, ind,  t_mex):
    
    mig_neand=get_migrating_tracts_ind(ts, 6, ind, t_mig) # 6 номер популяции Неандертальца
    
    
  
    mig_eu = get_migrating_tracts_ind(ts,1, ind, t_mex) #европейцы
    mig_na = get_migrating_tracts_ind(ts,2, ind, t_mex) # американцы(native americans)
    mig_af = get_migrating_tracts_ind(ts,0, ind, t_mex) # Африканцы   
        
    
    mig_eu.sort()
    mig_na.sort()
    mig_af.sort()    
    mig_neand.sort()

    
    return mig_neand, mig_eu, mig_na, mig_af

In [ ]:
time_units = 1000 / 25 #number of generations per 1000 years   
T = [600 * time_units, 70 * time_units, 30 * time_units,  time_units] #Old 300
t_mig = 50 * time_units 


N_e = 50000 
n = 250 
ind_number = 0
len_sequence = 12e7
t_mex = T[3]

ts, proportions = history_of_mexicans(len_sequence, T, t_mig, N_e, n)

In [ ]:
tractsND = get_migrating_tracts_ind(ts, 6, ind_number, t_mig)
tractsEU = get_migrating_tracts_ind(ts,1, ind_number, t_mex)
tractsAS = get_migrating_tracts_ind(ts,2, ind_number, t_mex)
tractsAF =get_migrating_tracts_ind(ts,0, ind_number, t_mex)

In [ ]:
cut = 1000

tractsND1 = clean_tracts(tractsND, cut)
tractsEU1 = clean_tracts(tractsEU, cut)
tractsAS1 = clean_tracts(tractsAS, cut)
tractsAF1 = clean_tracts(tractsAF, cut)

states_real = tracts_to_states(int(len_sequence/cut), tractsND1,tractsEU1,tractsAS1,tractsAF1)

In [ ]:
seqNd = createSeqObsS(ts, cut,ind_number,"NEAND")
seqEu = createSeqObsS(ts, cut,ind_number,"EU")
seqNa = createSeqObsS(ts, cut,ind_number,"NA")
seqAf = createSeqObsS(ts, cut,ind_number,"AF")
seq = np.column_stack((seqEu,seqNa,seqAf,seqNd))

number_states = [max(seqNd), max(seqEu), max(seqNa), max(seqAf)]


states = (0,1,2,3,4) # 0: Eu notNd, 1: Eu Nd, 2: NA notNd, 3: NA Nd, 4: Af


In [ ]:
S = initS(proportions[1], proportions[2], proportions[0])
A = initA(t_mig,T[3],2.5e-9, cut, proportions[1], proportions[2], proportions[0])
B = initB(1.25e-8,cut,t_mig,T[2],T[3],T[1],T[0], max(number_states)+1)


res = viterbi(seq, S, A, B)
tracts = get_HMM_tracts(res)
states = tracts_to_states_after_vit(int(len_sequence/cut),tracts)
print(sklearn.metrics.classification_report(states_real,states,  digits=5))
print(sklearn.metrics.confusion_matrix( states_real, states))